## Installation

In [1]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 5.4 MB/s eta 0:00:00


## API Key

In [2]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

## Define Main Chat Manager

In [16]:
"""
Module for managing conversations with Claude API using multiple tools.

This module provides a class-based interface for interacting with Claude's API,
maintaining conversation history, and handling multiple tool use requests dynamically.
Supports article search and web search tools with extensible architecture.
"""

from typing import List, Dict, Any, Optional, Callable
from abc import ABC, abstractmethod


class ToolHandler(ABC):
    """Abstract base class for tool handlers."""

    @abstractmethod
    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle the tool execution.

        Args:
            tool_input: The input parameters for the tool.

        Returns:
            str: The result of the tool execution.
        """


class ArticleSearchHandler(ToolHandler):
    """Handler for article search tool."""

    def __init__(self, get_article_func: Callable[[str], str]):
        """
        Initialize the article search handler.

        Args:
            get_article_func: Function to retrieve articles by search term.
        """
        self.get_article_func = get_article_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle article search requests.

        Args:
            tool_input: Dictionary containing 'search_term' key.

        Returns:
            str: The retrieved article content.

        Raises:
            KeyError: If 'search_term' is missing from tool_input.
        """
        search_term = tool_input["search_term"]
        print(f"Invoking tool: ... wait ... Searching for article: {search_term}")
        return self.get_article_func(search_term)


class WebSearchHandler(ToolHandler):
    """Handler for web search tool."""

    def __init__(self, web_search_func: Callable[[str], str]):
        """
        Initialize the web search handler.

        Args:
            web_search_func: Function to perform web searches by topic.
        """
        self.web_search_func = web_search_func

    def handle(self, tool_input: Dict[str, Any]) -> str:
        """
        Handle web search requests.

        Args:
            tool_input: Dictionary containing 'topic' key.

        Returns:
            str: The web search results.

        Raises:
            KeyError: If 'topic' is missing from tool_input.
        """
        topic = tool_input["topic"]
        print(f"Invoking tool: ... wait ... Searching the web for: {topic}")
        return self.web_search_func(topic)


class MultiToolChatManager:
    """
    Manager class for handling conversations with Claude API using multiple tools.

    This class maintains conversation history, handles multiple tool use requests,
    and provides a clean interface for interacting with Claude's API with support
    for article search, web search, and other extensible tools.
    """

    def __init__(
        self,
        client: Any,
        model: str = "claude-sonnet-4-20250514",
        max_tokens: int = 1000,
        tools: Optional[List[Dict[str, Any]]] = None
    ):
        """
        Initialize the multi-tool chat manager.

        Args:
            client: The Claude API client instance.
            model: The Claude model to use for conversations.
            max_tokens: Maximum tokens for each API response.
            tools: List of tool definitions available to Claude.
        """
        self.client = client
        self.model = model
        self.max_tokens = max_tokens
        self.tools = tools or []
        self.messages: List[Dict[str, Any]] = []
        self.tool_handlers: Dict[str, ToolHandler] = {}
        self._tool_call_count = 0  # Track number of tool calls for debugging

    def register_tool_handler(self, tool_name: str, handler: ToolHandler) -> None:
        """
        Register a handler for a specific tool.

        Args:
            tool_name: The name of the tool to handle.
            handler: The ToolHandler instance to process this tool.
        """
        self.tool_handlers[tool_name] = handler
        print(f"Registered handler for tool: {tool_name}")

    def get_registered_tools(self) -> List[str]:
        """
        Get list of all registered tool names.

        Returns:
            List[str]: Names of all registered tools.
        """
        return list(self.tool_handlers.keys())

    def get_history(self) -> List[Dict[str, Any]]:
        """
        Get the complete conversation history.

        Returns:
            List[Dict[str, Any]]: A copy of the current conversation history.
        """
        return self.messages.copy()

    def append_to_history(self, message: Dict[str, Any]) -> None:
        """
        Append a message to the conversation history.

        Args:
            message: The message dictionary to add to history.

        Raises:
            ValueError: If the message doesn't have required 'role' key.
        """
        if "role" not in message:
            raise ValueError("Message must contain 'role' key")

        self.messages.append(message)

    def clear_history(self) -> None:
        """Clear the entire conversation history and reset tool call counter."""
        self.messages.clear()
        self._tool_call_count = 0
        print("Conversation history cleared")

    def get_tool_call_count(self) -> int:
        """
        Get the number of tool calls made in the current conversation.

        Returns:
            int: Number of tool calls made.
        """
        return self._tool_call_count

    def _handle_tool_use(self, response: Any) -> Any:
        """
        Handle tool use requests from Claude's response.

        Args:
            response: The response object from Claude API.

        Returns:
            Any: The final response after tool processing.

        Raises:
            ValueError: If tool handler is not registered for the requested tool.
            AttributeError: If response object doesn't have expected attributes.
            IndexError: If response content is empty.
        """
        # Extract tool use information from Claude's response
        tool_use = response.content[-1]  # Get the last content item (tool use)
        tool_name: str = tool_use.name
        tool_input: Dict[str, Any] = tool_use.input

        # Increment tool call counter
        self._tool_call_count += 1

        print(f"... wait ... Claude wants to use tool: {tool_name} (Call #{self._tool_call_count})")

        # Add Claude's tool use request to the conversation history
        self.append_to_history({"role": "assistant", "content": response.content})

        # Check if we have a handler for this tool
        if tool_name not in self.tool_handlers:
            available_tools = ", ".join(self.get_registered_tools())
            raise ValueError(
                f"No handler registered for tool: {tool_name}. "
                f"Available tools: {available_tools}"
            )

        # Execute the tool using the registered handler
        try:
            tool_result: str = self.tool_handlers[tool_name].handle(tool_input)
            print(f"Tool {tool_name} executed successfully!")
        except Exception as e:
            # Handle tool execution errors gracefully
            tool_result = f"Error executing tool {tool_name}: {str(e)}"
            print(f"Error in tool {tool_name}: {str(e)}")

        # Construct the tool result message to send back to Claude
        tool_response: Dict[str, Any] = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,  # Match the tool use ID
                    "content": tool_result
                }
            ]
        }

        # Add the tool result to the conversation history
        self.append_to_history(tool_response)

        # Send the tool result back to Claude for final processing
        final_response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        return final_response

    def invoke_chat(self, question: str, print_response: bool = True) -> str:
        """
        Send a question to Claude and handle any tool use requests.

        This method supports recursive tool calls - Claude can use multiple tools
        in sequence to answer a single question.

        Args:
            question: The question or message to send to Claude.
            print_response: Whether to print the response to stdout.

        Returns:
            str: Claude's final response text.

        Raises:
            AttributeError: If the response object doesn't have expected attributes.
            IndexError: If the response content list is empty.
            ValueError: If a requested tool handler is not registered.
        """
        # Add the user's question to conversation history
        user_message = {"role": "user", "content": question}
        self.append_to_history(user_message)

        # Send the request to Claude with tool availability
        response = self.client.messages.create(
            model=self.model,
            messages=self.messages,
            max_tokens=self.max_tokens,
            tools=self.tools
        )

        # Handle tool use if Claude requested it (supports multiple tool calls)
        max_tool_iterations = 5  # Prevent infinite tool calling loops
        iteration_count = 0

        while response.stop_reason == "tool_use" and iteration_count < max_tool_iterations:
            response = self._handle_tool_use(response)
            iteration_count += 1

            if iteration_count >= max_tool_iterations:
                print(f"Warning: Reached maximum tool iterations ({max_tool_iterations})")
                break

        if response.stop_reason != "tool_use":
            # Claude provided a final answer
            if print_response and iteration_count == 0:
                print("Claude answered directly without using tools")
            elif print_response and iteration_count > 0:
                print(f"Claude completed answer after {iteration_count} tool call(s)")

        # Extract the final response text
        final_text = response.content[0].text

        # Add Claude's final response to history
        self.append_to_history({"role": "assistant", "content": response.content})

        if print_response:
            print("Claude's response:")
            print(final_text)

        return final_text

    def add_system_message(self, system_content: str) -> None:
        """
        Add a system message to the conversation history.

        Args:
            system_content: The system message content.
        """
        system_message = {"role": "system", "content": system_content}
        # Insert system message at the beginning if messages exist
        if self.messages:
            self.messages.insert(0, system_message)
        else:
            self.append_to_history(system_message)
        print("System message added to conversation")

    def remove_tool(self, tool_name: str) -> bool:
        """
        Remove a tool handler and its definition.

        Args:
            tool_name: The name of the tool to remove.

        Returns:
            bool: True if tool was removed, False if tool was not found.
        """
        if tool_name in self.tool_handlers:
            del self.tool_handlers[tool_name]
            # Remove from tools list
            self.tools = [tool for tool in self.tools if tool.get("name") != tool_name]
            print(f"Removed tool: {tool_name}")
            return True
        return False

    def add_tool_definition(self, tool_definition: Dict[str, Any]) -> None:
        """
        Add a new tool definition to the available tools.

        Args:
            tool_definition: The tool definition dictionary.

        Raises:
            ValueError: If tool definition is missing required fields.
        """
        required_fields = ["name", "description", "input_schema"]
        for field in required_fields:
            if field not in tool_definition:
                raise ValueError(f"Tool definition missing required field: {field}")

        # Check if tool already exists and replace it
        tool_name = tool_definition["name"]
        self.tools = [tool for tool in self.tools if tool.get("name") != tool_name]
        self.tools.append(tool_definition)
        print(f"Added tool definition: {tool_name}")


# Predefined tool definitions
ARTICLE_SEARCH_TOOL = {
    "name": "get_article",
    "description": "Search for and retrieve Wikipedia articles",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The term to search for in Wikipedia"
            }
        },
        "required": ["search_term"]
    }
}

WEB_SEARCH_TOOL = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic to search the web for"
            },
        },
        "required": ["topic"]
    }
}


def create_multi_tool_chat_manager(
    client: Any,
    get_article_func: Optional[Callable[[str], str]] = None,
    web_search_func: Optional[Callable[[str], str]] = None,
    include_article_search: bool = True,
    include_web_search: bool = True
) -> MultiToolChatManager:
    """
    Create a configured MultiToolChatManager with multiple tools.

    Args:
        client: The Claude API client instance.
        get_article_func: Function to retrieve articles by search term.
        web_search_func: Function to perform web searches by topic.
        include_article_search: Whether to include article search tool.
        include_web_search: Whether to include web search tool.

    Returns:
        MultiToolChatManager: Configured chat manager with requested tools.

    Raises:
        ValueError: If a tool is requested but its function is not provided.
    """
    # Prepare tools list
    tools = []

    if include_article_search:
        if get_article_func is None:
            raise ValueError("get_article_func must be provided when include_article_search=True")
        tools.append(ARTICLE_SEARCH_TOOL)

    if include_web_search:
        if web_search_func is None:
            raise ValueError("web_search_func must be provided when include_web_search=True")
        tools.append(WEB_SEARCH_TOOL)

    # Create the chat manager
    chat_manager = MultiToolChatManager(
        client=client,
        tools=tools
    )

    # Register tool handlers
    if include_article_search and get_article_func:
        article_handler = ArticleSearchHandler(get_article_func)
        chat_manager.register_tool_handler("get_article", article_handler)

    if include_web_search and web_search_func:
        web_handler = WebSearchHandler(web_search_func)
        chat_manager.register_tool_handler("web_search", web_handler)

    print(f"Multi-tool chat manager created with {len(tools)} tool(s)")
    return chat_manager


def create_article_only_chat_manager(
    client: Any,
    get_article_func: Callable[[str], str]
) -> MultiToolChatManager:
    """
    Create a chat manager with only article search capability.

    Args:
        client: The Claude API client instance.
        get_article_func: Function to retrieve articles by search term.

    Returns:
        MultiToolChatManager: Chat manager with article search tool only.
    """
    return create_multi_tool_chat_manager(
        client=client,
        get_article_func=get_article_func,
        include_web_search=False
    )


def create_web_only_chat_manager(
    client: Any,
    web_search_func: Callable[[str], str]
) -> MultiToolChatManager:
    """
    Create a chat manager with only web search capability.

    Args:
        client: The Claude API client instance.
        web_search_func: Function to perform web searches by topic.

    Returns:
        MultiToolChatManager: Chat manager with web search tool only.
    """
    return create_multi_tool_chat_manager(
        client=client,
        web_search_func=web_search_func,
        include_article_search=False
    )

## Define Tools

Define custom tool: `get_article` a function to search items from wikipedia

In [5]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=1f5a86415bb5280d7d6cf601d246cc97424b119f1d8ac38049521ff337d6c918
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [17]:
"""Module for retrieving Wikipedia article content."""

import wikipedia


def get_article(search_term: str) -> str:
    """
    Retrieve the content of a Wikipedia article based on a search term.

    Args:
        search_term (str): The term to search for on Wikipedia.

    Returns:
        str: The full text content of the Wikipedia article.

    Raises:
        IndexError: If no search results are found.
        wikipedia.exceptions.DisambiguationError: If the search term is ambiguous.
        wikipedia.exceptions.PageError: If the page does not exist.
    """
    # Search Wikipedia for articles matching the search term
    results = wikipedia.search(search_term)

    # Get the first result from the search
    first_result = results[0]

    # Retrieve the Wikipedia page object for the first result
    page = wikipedia.page(first_result, auto_suggest=False)

    # Return the full text content of the article
    return page.content

In [7]:
pip install serpapi google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=46659bd41029900afdae6733487ba561ef17baa27285634309310c03b3eca105
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [20]:
from google.colab import userdata
from serpapi import GoogleSearch
from typing import List, Dict, Any

def search_serpapi(query: str) -> List[Dict[str, Any]]:
    """
    Search using SerpAPI for the given query and return the results.

    :param query: The search query string.
    :return: A list of search results.
    :raises Exception: For any errors during the request.
    """
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
    try:
        search = GoogleSearch({
            "q": query,
            "location": "New York, NY, United States",
            "api_key": SERPAPI_API_KEY
        })
        raw_results = search.get_dict()
        results = raw_results.get("organic_results", [])
        snippet = ' '.join([results[i]['snippet'] for i in range(len(results))])
        return snippet
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

## Define Client

In [9]:
from anthropic import Anthropic

client = Anthropic(api_key=CLAUDE_API_KEY)

## Invoke

In [21]:
# Create manager with both tools
chat_manager = create_multi_tool_chat_manager(
    client=client,
    get_article_func=get_article,
    web_search_func=search_serpapi
)

# Use the manager
response = chat_manager.invoke_chat("What's the latest news about Tesla?")

# Check what tools were used
print(f"Tools used: {chat_manager.get_tool_call_count()}")
print(f"Available tools: {chat_manager.get_registered_tools()}")

Registered handler for tool: get_article
Registered handler for tool: web_search
Multi-tool chat manager created with 2 tool(s)
... wait ... Claude wants to use tool: web_search (Call #1)
Invoking tool: ... wait ... Searching the web for: Tesla latest news
Tool web_search executed successfully!
Claude completed answer after 1 tool call(s)
Claude's response:
Based on the latest search results, here are some of the key recent developments about Tesla:

## Recent Tesla News:

**New Product Launch:**
- Tesla is launching the new **Model 3 Performance**, described as a highly differentiated performance trim that uses Tesla's latest manufacturing and engineering capabilities

**Autonomous Driving Progress:**
- A Tesla vehicle **delivered itself to a customer autonomously**, as confirmed by Elon Musk
- Tesla's **robotaxi service** is sending out more invites for next phases, though first impressions have been mixed due to some operational mistakes
- Tesla has hired **Henry Kuang**, former aut

## Continuous Chat Interface

In [23]:
while True:
    user_input = input("> You: ")

    if user_input.lower() == 'exit' or user_input.lower() == 'quit':
        break

    # Create manager with both tools
    chat_manager = create_multi_tool_chat_manager(
        client=client,
        get_article_func=get_article,
        web_search_func=search_serpapi
    )

    # Use the manager
    response = chat_manager.invoke_chat(user_input)

    # Check what tools were used
    print(f"... Tools used: {chat_manager.get_tool_call_count()}")
    print(f"... Available tools: {chat_manager.get_registered_tools()}")
    print(f"> Claude: {response}")

> You: tom holland
Registered handler for tool: get_article
Registered handler for tool: web_search
Multi-tool chat manager created with 2 tool(s)
... wait ... Claude wants to use tool: get_article (Call #1)
Invoking tool: ... wait ... Searching for article: Tom Holland
Tool get_article executed successfully!
Claude completed answer after 1 tool call(s)
Claude's response:
Based on the Wikipedia article, here's what I found about Tom Holland:

**Tom Holland** (born June 1, 1996) is an English actor who has become one of the most popular actors of his generation. Here are the key highlights:

## Career Highlights:
- **Spider-Man**: Best known for playing Peter Parker/Spider-Man in the Marvel Cinematic Universe, starting with *Captain America: Civil War* (2016)
- **Record holder**: Youngest actor to play a title role in an MCU film (*Spider-Man: Homecoming*)
- **Box office success**: His Spider-Man films have been hugely successful, with *Spider-Man: No Way Home* (2021) becoming the highe